# Download dataset, import libraries, data cleaning

In [1]:
"""
Code for downloading dataset, unzipping, and deleting the zip file
If running the code, make sure you have a Kaggle account and you have a public API token saved to a .kaggle folder in your 'User' folder
Details -> https://www.kaggle.com/docs/api
Otherwise just clone the Github repo with the csv file already downloaded and extracted
"""

# from zipfile import ZipFile
# import os
# !pip install kaggle
# !kaggle datasets download -d matheusfonsecachaves/popular-video-games
# with ZipFile(os.path.join(os.getcwd(), 'popular-video-games.zip'), 'r') as zip_file:
#     zip_file.extractall(os.getcwd())
# os.remove('popular-video-games.zip')

"\nCode for downloading dataset, unzipping, and deleting the zip file\nIf running the code, make sure you have a Kaggle account and you have a public API token saved to a .kaggle folder in your 'User' folder\nDetails -> https://www.kaggle.com/docs/api\nOtherwise just clone the Github repo with the csv file already downloaded and extracted\n"

In [2]:
# Library imports
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval as string_to_list
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
import os
from torch.nn import functional as F

c:\Users\Lucas\anaconda3\envs\Information_retrieval\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
"""
Read csv file and display missing values & data types
"""
games = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'backloggd_games.csv'), index_col=0)
print('Dataset Missing Values & data types')
games.info()

"""
1. Convert numerical columns formatted as strings to floats with the full value
2. Convert the list columns represented as strings into actual lists
3. Fill missing summary values with an empty string
4. Drops duplicates -> Keeps the first value
"""
def string_to_int(x):
    if 'K' in x:
        x = float(x.replace('K','')) * 1000
    else:
        x = float(x) * 1000
    return x

games[['Plays','Playing','Backlogs','Wishlist','Lists','Reviews']] = games[['Plays','Playing','Backlogs','Wishlist','Lists','Reviews']].map(string_to_int)
games[['Developers','Platforms','Genres']] = games[['Developers','Platforms','Genres']].map(string_to_list)
games['Summary'] = games['Summary'].fillna('')
games = games.drop_duplicates(subset='Title')

"""
Create the document list from the specified fields
"""
games['Documents'] =    games['Title'] + ' [SEP] ' +                          \
                        'Release_Date: ' + games['Release_Date'] + ' [SEP] ' +                   \
                        'Developers: ' + games['Developers'].str.join(' ') + ' [SEP] ' +       \
                        'Genres: ' + games['Genres'].str.join(' ') + ' [SEP] ' +                        \
                        'Platforms: ' + games['Platforms'].str.join(' ') + ' [SEP] ' +        \
                        games['Summary']
documents = games['Documents'].to_list()

# Print first two documents
for document in documents[:2]: print(document + '\n')

Dataset Missing Values & data types
<class 'pandas.core.frame.DataFrame'>
Index: 60000 entries, 0 to 59999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         60000 non-null  object 
 1   Release_Date  60000 non-null  object 
 2   Developers    60000 non-null  object 
 3   Summary       55046 non-null  object 
 4   Platforms     60000 non-null  object 
 5   Genres        60000 non-null  object 
 6   Rating        25405 non-null  float64
 7   Plays         60000 non-null  object 
 8   Playing       60000 non-null  object 
 9   Backlogs      60000 non-null  object 
 10  Wishlist      60000 non-null  object 
 11  Lists         60000 non-null  object 
 12  Reviews       60000 non-null  object 
dtypes: float64(1), object(12)
memory usage: 6.4+ MB
Elden Ring [SEP] Release_Date: Feb 25, 2022 [SEP] Developers: FromSoftware Bandai Namco Entertainment [SEP] Genres: Adventure RPG [SEP] Platforms: Windows PC Play

# BERT

In [4]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else 'cpu')
print('using', device)
model.to(device)
model.eval()

def encode_texts(texts, batch_size=64, pooling='mean', max_length=None):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(
            batch_texts, 
            add_special_tokens=True,
            return_tensors='pt', 
            truncation=True, 
            padding='max_length', 
            max_length=max_length
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
        
        if pooling == 'mean':
            token_embeddings = outputs.last_hidden_state  # [batch_size, seq_length, hidden_size]
            mask = inputs["attention_mask"].unsqueeze(-1).expand(token_embeddings.size()).float()
            batch_embeddings = torch.sum(token_embeddings * mask, dim=1) / torch.clamp(mask.sum(dim=1), min=1e-9)
        elif pooling == 'mean-w/o-mask':
            token_embeddings = outputs.last_hidden_state
            batch_embeddings = torch.mean(token_embeddings, dim=1)
        else:
            # Use the pooled output ([CLS] token) for each document in the batch.
            batch_embeddings = outputs.pooler_output  # shape: [batch_size, hidden_size]
        all_embeddings.append(batch_embeddings)
    # Concatenate all batches into one tensor.
    return torch.cat(all_embeddings, dim=0)
def encode_text(text, pooling='mean', max_length=None):
    """
    Encodes text into an embedding using BERT.
    Using a larger max_length to accommodate longer documents.
    """
    inputs = tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', truncation=True, padding='max_length', max_length=max_length)
    # Move inputs to the correct device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    if pooling == 'mean':
        token_embeddings = outputs.last_hidden_state[0]
        mask = inputs["attention_mask"][0].unsqueeze(-1).expand(token_embeddings.size()).float()
        mean_pool = torch.sum(token_embeddings * mask, dim=0) / torch.clamp(mask.sum(dim=0), min=1e-9)
        return mean_pool
    elif pooling == 'mean-w/o-mask':
        return torch.mean(outputs.last_hidden_state[0], dim=0)
    else:
        return outputs.pooler_output[0]

def generate_document_embeddings(documents, pooling='mean', max_length=512, save_path='test.pt', device='cpu'):
    if os.path.exists(save_path):
        print("Loading precomputed document embeddings.")
        return torch.load(save_path).to(device)
    
    doc_embeddings = encode_texts(documents, pooling=pooling, max_length=max_length)
    torch.save(doc_embeddings, save_path)
    return doc_embeddings.to(device)

doc_embeddings = generate_document_embeddings(documents, pooling='mean-w/o-mask', max_length=256, save_path='doc_embeddings_wo_mask.pt', device=device)
def get_top_k_documents(query, top_k=40, pooling='mean'):
    query_embedding = encode_text(query, pooling=pooling, max_length=256).unsqueeze(0)

    similarities = F.cosine_similarity(query_embedding, doc_embeddings, dim=1)
    top_k_indices = torch.topk(similarities, k=top_k).indices
    top_results = [(documents[i], similarities[i].item(), i) for i in top_k_indices]

    return top_results, similarities

using cuda
Loading precomputed document embeddings.


In [5]:
results, similarities = get_top_k_documents('zelda', pooling='mean', top_k=40)
for item in results:
    print(item)

print(similarities[:5])

('Zelda: The Wand of Gamelon [SEP] Release_Date: Oct 10, 1993 [SEP] Developers: Philips Interactive Media, Inc. Animation Magic [SEP] Genres: Adventure Puzzle [SEP] Platforms: Philips CD-i [SEP] Zelda: The Wand of Gamelon was the product of a compromise between Nintendo and Philips following their failure to release a CD-ROM based add-on to the Super Nintendo Entertainment System. It was developed in tandem with Link: The Faces of Evil and marks the first game where the eponymous princess Zelda is playable. The Wand of Gamelon differ from most conventional Zelda games as it is a platformer with a side-scrolling view, similar to The Adventure of Link. Stages are accessed from a world map, with more becoming available as Zelda clears an area or defeats a boss. The game have been subject to much criticism and Nintendo does not recognize it as part of the series.', 0.5395450592041016, tensor(36018, device='cuda:0'))
('Wizardry Gaiden III: Scripture of the Dark [SEP] Release_Date: Oct 25, 1

## Generate Results

In [6]:
queries = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'queries.csv'))
queries = queries['Query'].to_list()
data = []
for query in queries:
    print(f"Query: {query}")
    results, similarities = get_top_k_documents(query, pooling='mean', top_k=40)
    for item in results:
        data.append({'Query': query, 'Title': games.iloc[item[2].item()]['Title'], 'ID': item[2].item(), 'Similarity': item[1]})

Query: the witcher 3
Query: Wild hunt
Query: Botw
Query: final fantasy
Query: Monster hunter
Query: Upcoming indie platformers for PC
Query: Top-rated multiplayer FPS games on PlayStation 5
Query: Best open-world RPG titles for Xbox Series X
Query: Story-driven adventure games with strong plot
Query: Kid-friendly Switch games for family play
Query: Turn-based strategy games released in the last year
Query: Retro arcade games with modern ports
Query: Local co-op platformers on Steam
Query: Best VR-exclusive games on Oculus Quest
Query: Top survival horror games of all time
Query: Open-world fantasy games like The Elder Scrolls
Query: Recommended 2D fighters for beginner players
Query: High-rated indie roguelike deckbuilders
Query: Free-to-play battle royale games for mobile
Query: Remastered classic JRPGs on current-gen consoles
Query: Popular city-building simulation games
Query: Co-op puzzle-platformers for Nintendo Switch
Query: Easygoing life sims for relaxing gameplay
Query: Hardco

In [7]:
result_df = pd.DataFrame(data)
result_df = pd.merge(result_df, games.drop(columns=['Documents']), on='Title', how='left')
result_df.to_csv('results_bert_wo_mask.csv', index=False)

# TF-IDF

In [8]:
# Ensure games_df is the filtered one (same as used to build `documents`)
games_df = games.reset_index(drop=True)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words="english", max_features=10000)
doc_tfidf = vectorizer.fit_transform(documents)
query_tfidf = vectorizer.transform(queries)

# Compute cosine similarity between queries and documents
similarity_scores = cosine_similarity(query_tfidf, doc_tfidf)

# Prepare top 40 results
results = []
top_k = 40

for query_idx, (query_text, scores) in enumerate(zip(queries, similarity_scores)):
    top_indices = scores.argsort()[::-1][:top_k]
    
    for game_idx in top_indices:
        game = games_df.iloc[game_idx]
        results.append({
            "Query": query_text,
            "Title": game["Title"],
            "ID": game.name,
            "Similarity": scores[game_idx],
            "Release_Date": game["Release_Date"],
            "Developer": " | ".join(game["Developers"]),
            "Summary": game["Summary"],
            "Platforms": " | ".join(game["Platforms"]),
            "Genres": " | ".join(game["Genres"]),
            "Rating": game["Rating"],
            "Plays": game["Plays"],
            "Playing": game["Playing"],
            "Backlogs": game["Backlogs"],
            "Wishlist": game["Wishlist"],
            "Lists": game["Lists"],
            "Reviews": game["Reviews"]
        })

## Generate Results

In [9]:
# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("results_tfidf.csv", index=False)

print("TF-IDF results saved to results_tfidf.csv.")

TF-IDF results saved to results_tfidf.csv.


# BM-25

In [10]:
def BM_25_matrix_creation(documents, k_1, b, max_features=50000, min_df=2):
    """
    Pre-computes the BM-25 matrix as a sparse matrix. Having it as a normal matrix makes the computation really expensive
    """

    # Defines a pipeline where the first transformation creates the term-document count matrix and the second transforms it into the TF-IDF matrix
    # The pipeline is needed to have access to the term document count matrix 
    pipe = Pipeline([('count', CountVectorizer(max_features=max_features, min_df=min_df)), ('tfid', TfidfTransformer())]).fit(documents)
    term_doc_matrix = pipe['count'].transform(documents)

    # Defines the variables used in the BM-25 formula
    doc_lengths, avg_dl, idfs, tfs = term_doc_matrix.sum(axis=1), np.mean(term_doc_matrix.sum(axis=1)), pipe['tfid'].idf_.reshape(1, -1), term_doc_matrix.multiply(1 / term_doc_matrix.sum(axis=1))

    # Calculates the BM-25 matrix
    # .multiply is the scipy sparse matrix multiplication method 
    numerator = (k_1 + 1) * tfs
    denominator = k_1 * ((1 - b) + b * (doc_lengths / avg_dl)) + tfs
    BM25 = numerator.multiply(1 / denominator)
    BM25 = BM25.multiply(idfs)

    # Maps each of the terms in the vocabulary to an index
    vocab = pipe['count'].get_feature_names_out()
    vocab = {term:index for index, term in enumerate(vocab)}
    
    return BM25.tocsr(), vocab # A CSR matrix enables matrix slicing and indexing

# max_features = maximum number of terms, min_df = number of occurrences required to include the term in the matrix
BM25, vocab = BM_25_matrix_creation(documents, k_1=1.2, b=0.8, max_features=50000, min_df=1)

## Generate Results

In [11]:
def retrieve_top_k(query, BM25_matrix, vocabulary, doc_titles, epsilon=1e-6, top_k=40):
    """
    Retrieves the top k documents from a singular given query
    """

    # Returns the term indices for the terms in the query or OOV (Out-Of-Vocab) if the term is not found in vocab
    query_tokens = [vocabulary[term] if term in vocabulary else 'OOV' for term in query]

    # Returns only the In-Vocab tokens
    IV = [term for term in query_tokens if term != 'OOV']


    if len(IV) != 0:

        # Filters the BM25 matrix to only the In-Vocab terms and sums the scores across each document
        doc_scores = BM25_matrix[:, IV].sum(axis=1)
        if 'OOV' in query:
            doc_scores += np.full((BM25_matrix.shape[0], 1), epsilon) # Adds a small epsilon constant if 'OOV' is in the query

    else:
        doc_scores = np.full((BM25_matrix.shape[0], 1), epsilon) # Defaults to the small epsilon constant if no IV terms in the query

    # Flattens the doc scores to a 1D array
    doc_scores = np.ravel(doc_scores)
    
    # Ranked docs are returned in the format -> (doc_ID, doc_title), score
    ranked_docs = sorted(zip(enumerate(doc_titles), doc_scores), key=lambda zipper: zipper[1], reverse=True)
    top_k_docs = ranked_docs[:top_k]

    return top_k_docs

queries = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'queries.csv'))
queries = [x.split() for x in queries['Query'].to_list()]

# Retrieves results for each of the queries and reformats the results -> (query, doc_title, doc_ID, score)
results = []
for query in queries:
    top_k = retrieve_top_k(query, BM25, vocab, games['Title'])
    top_k = [(' '.join(query), k[0][1], k[0][0], k[1]) for k in top_k]
    for k in top_k: results.append(k)

# Saves the results to a dataframe with the full document data and exports as a csv
results_df = pd.DataFrame(results, columns=['Query','Title','ID','Similarity'])
results_df = pd.merge(results_df, games.drop(columns=['Documents']), how='left', on='Title')
results_df.to_csv('results_BM25.csv', index=False)

# Result Pooling

In [12]:
# Concatenates the three result csv files then saves the result as a new csv
bert_results, bm25_results, tfidf_results = pd.read_csv('results_bert_wo_mask.csv'), pd.read_csv('results_BM25.csv'), pd.read_csv('results_tfidf.csv')
unpooled_results = pd.concat([bert_results, bm25_results, tfidf_results], axis=0, ignore_index=True)
unpooled_results.to_csv('results_UNPOOLED.csv', index=False)

# Removes the duplicates according to the query and ID -> Only unique retrieved documents for each query
pooled_results = unpooled_results.drop_duplicates(subset=['Query','ID'], ignore_index=True)
pooled_results.to_csv('results.csv', index=False)